# EE 120 Lab 2: The Fast Fourier Transform (FFT) and its Applications
v1 - Fall 2018: Dominic Carrano  
v2 - Spring 2019: Dominic Carrano, Sidhant Kaushik, Shubhrakanti Ganguly, Miki Lustig

# Installations

You'll need to install [sk-video](https://pypi.org/project/sk-video/) and [FFmpeg](https://www.ffmpeg.org/download.html) for the video processing we'll do in Question 4. You need to do this before the first cell below will run.

In [ ]:
from math import log
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import timeit, time
import skvideo.io  
%matplotlib inline

# Background

The background section for this lab is mostly focused on how the DFT ties into the other Fourier analysis methods you've learned (DTFS, DTFT) rather than the actual FFT algorithm. The algorithm is discussed within Question 2 in great detail. The FFT is, according to MIT math professor Gilbert Strang, "the most important numerical algorithm of our lifetime", and you'll find plenty of resources online about it beyond what we provide here. We've linked a few of what we consider to be the most useful ones in the References section at the end.

## DFT related to DTFS

Living in the discrete world, we have 2 Fourier-based tools available to us: the DTFS and DTFT. Again, the DTFT of a signal is a continuous (albeit $2\pi$ periodic) waveform, so we can't use it directly for computation as is - it has an infinite number of data points, which we can't store on a computer. 

What we use is related to both, and is known as the **Discrete Fourier Transform (DFT)**. The DFT takes a length $N$ vector $x(n)$ (aka our time-domain signal) and returns back a length $N$ vector $X[k]$ (also a "signal", but in the frequency domain). On your homework, you showed that the DTFS and DFT are very intimately related - the scaling factor is just moved from the analysis equation (in the DTFS) to the synthesis equation (in the DFT). 

We only consider the DTFS of periodic signals, and so you may wonder why it's theoretically justified to take the DFT of any old dataset we have on our computer, since it's just a scaled DTFS. One way to reason about this is that we assume that our data truly is periodic, and what we have here represents one period of it. The other interpretation, which is how the DFT is related to the DTFT, providing another way to reason about what it means to take the DFT of a given array of data, is discussed in the next section.

## DFT related to DTFT

The DFT (for an input of size $N$), it turns out, is actually just the DTFT sampled at $N$ uniformly spaced points for one period of the DTFT, which is one of the reasons it's so nice to use. Let's look at the analysis (the ones that take us from time into frequency) equation for each transform to gain some idea of why this is true.

Assuming our discrete-time signal is $x(n)$, we have its DFT $X[k]$ and DTFT $X(\omega)$ given by:

$$X[k] = \sum_{n = 0}^{N - 1} x(n)e^{-i\frac{2\pi}{N}kn}\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ X(\omega) = \sum_{n = -\infty}^{\infty} x(n)e^{-i\omega n}$$

But we said our signal $x(n)$ is only of duration $N$, and we essentially disregard any other samples (in theory land, we allow for infinite-duration signals, but again, we're dealing with computers with finite memory here, so just being discrete instead of continuous is not enough - the signal must also be finite-duration). In that case we can rewrite the summation of our DTFT from $0$ to $N - 1$:

$$X(\omega) = \sum_{n = 0}^{N - 1} x(n)e^{-i\omega n}$$

Now, we've said that the DFT is just a sampling of the DTFT, so each $k$ corresponds to a different sample. Then we're taking $N$ samples from a $2\pi$ length interval, so each one should be spaced a distance $\frac{2\pi}{N}$ apart, and then $\frac{2\pi}{N}k$ should be the frequency that the $k$th DFT "bin" grabs. And indeed, if we set $\omega = \frac{2\pi}{N}k$ for the DTFT equation, we now get:

$$X\bigg(\frac{2\pi}{N}k\bigg) = \sum_{n = 0}^{N - 1} x(n)e^{-i\frac{2\pi}{N}k n}$$

This is our DTFT parameterized in terms of $\frac{2\pi}{N}k$ (remember that $N$ is some fixed constant - the variable here is $k$). And since we can only access samples at these intervals, we re-index the left hand side to be purely in terms of k (so it's now discretized, and we'll switch back from parentheses to square brackets in terms of notation), giving us the DFT:

$$X[k] = \sum_{n = 0}^{N - 1} x(n)e^{-i\frac{2\pi}{N}k n}$$

This is the best way to think about the DFT: it's a uniform sampling of one period of the DTFT. From this, various properties become obvious. For example, the longer a DFT we take (by padding the signal with zeros), the better approximation of the DTFT we get.

# Q1a: Naive DFT Code

We'll begin by implementing the DFT the "naive" way by simply translating the DFT analysis equation into code. Recall that if we have a length $N$ vector of data (or, in a computer, a numpy array of length $N$), the DFT analysis equation returns a sampling of $N$ points of one-period of its DTFT according to:

$$X[k] = \sum_{n = 0}^{N-1} x(n) e^{-i\frac{2\pi}{N}nk}$$

for $k, n \in \{0, 1, 2, ..., N - 1\}$. In general, $X[k]$ is a complex-valued array, even if $x(n)$ is purely real, which we must account for when writing our code by declaring our output type as `np.complex128` (a pair of floating-point "real" numbers, each 64 bits). If we don't, numpy will throw out the imaginary part when doing our computations, and return the incorrect result.

Fill in the function `dft` below to compute the DFT based on the analysis equation. In your array `X` that you output, `X[k], k = 0, 1, ..., N-1` should be the value of the `k`th DFT coefficient.

**Hint 1:** The function call `np.exp(z)` returns $e^z$, where $z$ is any number, real or complex.  
**Hint 2:** In Python, `-1j` is used to represent the complex number $i$. For example, `-1j * 2 * np.pi` would be $2\pi i$.

In [ ]:
def dft(x):
    N = len(x)
    X = np.zeros(N, dtype=np.complex128) # output array
    ## TODO ##

    
    ## TODO ##
    return X

Let's do a couple quick sanity checks on our naive DFT function to make sure it returns the correct results. Run the cells below to check your work. If all tests do not print "True" for whether or not they passed, your `dft` function is buggy. 

Don't worry if it takes around half a minute (it shouldn't take more than 3-4 minutes to run all 5 of the tests) for the fourth and fifth tests to run - it's because we wrote a naive implementation of the DFT, which takes a long time for large inputs! By the end of this notebook, we'll have made big improvements, and hopefully these tests will make you appreciate the FFT a bit, as 1024 and 2048 are fairly commonly used data chunk sizes to take FFTs of in the real world.

In [ ]:
def random_complex_array(N):
    """
    Generates a length N numpy array of complex numbers whose real and imaginary 
    parts are both chosen uniformly at random from the interval [0, 1).
    """
    re = np.random.rand(N)
    im = np.random.rand(N)
    return re + 1j*im

In [ ]:
def run_fft_tests(my_fft):
    """
    Run tests comparing FFT functions f, g, and display information about
    pass/fail and runtime. This is not meant to be used for performance
    profiling, rather just a ballpark for whether or not the function's
    runtimes are reasonable, as the time will also include printing time
    which technically shouldn't be factored in.
    """
    ref_fft = np.fft.fft
    t0 = time.time()
    
    # three tests on fixed data
    x1 = np.array([1, 2, 3])
    print("Test 1 passed: {0}".format(np.allclose(my_fft(x1), ref_fft(x1))))

    x2 = np.array([-1, 1j, -1, 1j])
    print("Test 2 passed: {0}".format(np.allclose(my_fft(x2), ref_fft(x2))))

    x3 = np.zeros(2 ** 8)
    print("Test 3 passed: {0}".format(np.allclose(my_fft(x3), ref_fft(x3))))

    # randomized larger datasets to test on
    x4 = np.random.rand(2 ** 12)
    print("Test 4 passed: {0}".format(np.allclose(my_fft(x4), ref_fft(x4))))

    x5 = random_complex_array(2 ** 12)
    print("Test 5 passed: {0}".format(np.allclose(my_fft(x5), ref_fft(x5))))
    
    tf = time.time()
    print("Tests took {0} seconds".format(round(tf - t0, 3)))

This will take anywhere from 30 seconds to a few minutes to run depending on your laptop's processing power. This cell isn't graded on time, just correctness.

In [ ]:
run_fft_tests(dft)

# Q1b: Naive DFT (Analysis)

Now that we've written our naive DFT function, let's do some basic performance analysis on it. Answer the questions below. At the end of the lab, we'll actually plot the runtime as a function of the input size, comparing our functions against the state of the art, but that will be fairly computationally intensive, so we'll just do the "pen and paper" analysis for now.

This part can be done entirely by inspecting the DFT analysis equation.

**Q1b, i: How many complex multipications are performed by `dft` in computing a single DFT coefficient for a size $N$ input? Multiplying two complex numbers counts as a single multiplication. For example, $x(1) \cdot e^{-i\frac{2\pi}{N}}$ counts as one complex multiplication, as each element of $x(n)$ is a complex number, in general.**  
<span style="color:blue">A: (TODO).</span>

**Q1b, ii: How many complex additions (adding two complex numbers) are performed by `dft` in computing a single DFT coefficient for a size $N$ input?**   
<span style="color:blue">A: (TODO)</span>

**Q1b, iii: Adding up your previous two answers,
how many complex number operations (complex additions and complex multiplications) are performed in calculating a single DFT coefficient for an input of size $N$?**  
<span style="color:blue">A: (TODO)</span>

**Q1b, iv: Multiplying your answer to (iii) by the number of coefficients, $N$, write the total number of complex number operations performed in calculating *all* $N$ of the DFT coefficients**:  
<span style="color:blue">A: (TODO)</span>

**Q1b, v: Asymptotically (i.e., in Big-O notation), what is the runtime of the function `dft` for naive calculation of the DFT? (If you have never seen Big-O notation, please post on Piazza or come to Office Hours)**:  
<span style="color:blue">A: (TODO)</span>

# Q2: The Fast Fourier Transform

## Background

Now that we've implemented the DFT the slow way, we'll do it the faster way: the Fast Fourier Transform (FFT).

### Decimation In Time vs. Decimation In Frequency

There are two main types of FFT algorithms: *Decimation In Time* (DIT), and *Decimation In Frequency* (DIF). They both exploit symmetries in computation to reduce the number of stages of computation from $O(N)$ to $O(\log_2(N))$. The DIT algorithm does so by decomposing $x(n)$ into successively smaller subsequences. The DIF approaches the problem by decomposing $X[k]$ into successively smaller subsequences. 

In this lab, we will implement the decimation in *time* algorithm. 

### Zero-Padding

This algorithm assumes that $N$, the length of $x(n)$, is a power of 2, and exploits symmetry in computations with an eye toward decomposing things in terms of powers of two.

There are two ways to handle the input length not being a power of 2:
1. Zero-pad to the nearest power of two. Zero-padding the input just results in a finer sampling of the DTFT - recall from the background section at the top of the lab that a higher $N$ implies more points that you sample of the $2\pi$ interval, since the DFT just samples the DTFT at $N$ uniformly spaced points. In fact, zero padding is often done to achieve this finer sampling of the DTFT beyond the amount required for computational efficiency. For example, you might zero-pad a length 926 signal to length 2048 even though 1024 would work to get the efficiency boost, purely to see a finer interpolation of the spectrum.
2. Call `dft`, your naive DFT routine, so that you get a DFT of the exact requested length, at the cost of more computation.

We'll take the second approach here. This gives anyone who uses our FFT function more fine-grained control—if they want to zero pad, they can do it themselves.

## The Algorithm

The core concept of the Decimation in Time FFT algorithm is the rearranging of the DFT of the signal $x(n)$ into two parts: a sum over the even-numbered indices $n={2m}$ and a sum over the odd-numbered indices $n={2m+1}$. Starting with the analysis equation,


$$
\begin{align*}
    X[k] &= \sum_{n = 0}^{N-1} x(n) e^{-i\frac{2\pi}{N}nk} \\
    &= \underbrace{\sum_{m = 0}^{N/2-1} x(2m) e^{-i\frac{2\pi}{N}(2m)k}}_\text{Even-indexed components of $x$} + \underbrace{\sum_{m = 0}^{N/2-1} x(2m + 1) e^{-i\frac{2\pi}{N}(2m + 1)k}}_\text{Odd-indexed components of $x$} \\
    &= \underbrace{\sum_{m = 0}^{N/2-1} x(2m) e^{-i\frac{2\pi}{N/2}mk}}_\text{Length $N/2$ DFT} + \underbrace{e^{-i\frac{2\pi}{N}k}}_\text{Scaling factor}\underbrace{\sum_{m = 0}^{N/2-1} x(2m + 1) e^{-i\frac{2\pi}{N/2}mk}}_\text{Length $N/2$ DFT} \\
    &= E[k] + e^{-i\frac{2\pi}{N}k} O[k]
\end{align*}
$$

where $E[k]$ is the $N/2$ length DFT of the signal $e[n] = [x[0], x[2], x[4], ..., x[N / 2]]$ and $O[k]$ the $N/2$ length DFT of the signal $o[n] = [x[1], x[3], x[5], ..., x[N / 2 - 1]]$. 

### Runtime

We've split the original size $N$ DFT into two DFTs of length $N/2$, one on the odd-index values, and one on the even-indexed values. So, we continually split our problem size in half, until we get to a size 1 input. In total, we can divide a number by 2 exactly $\log_2(N)$ times (assuming $N$ is a power of 2, which we do here), so we have $\log_2(N)$ stages of computation. 

In each stage, we have do one complex multiplication, $e^{-i\frac{2\pi}{N}k} O[k]$, and one complex addition, $E[k] + e^{-i\frac{2\pi}{N}k} O[k]$, per coefficient $X[k]$ to combine the recursively computed DFTs. So we do two complex operations per coefficient, and have $N$ coefficients, leading to $2N$ complex operations in total to combine the results. So, at each stage, we do a linear amount of work, and we have a logarithmic number of stages. The total work (asymptotically) done is thus a linear term times a logarithmic one, giving $O(N \log N)$ runtime.

### Visualization


This is often visualized through the following "butterfly diagram", shown here for $N=8$, named for the way that the multiplications at the end are rectified with lines crossing over each other like butterfly wings:

<br>  

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cb/DIT-FFT-butterfly.png" width="600">

<br>

Most literature on the FFT uses the shorthand notation $W_N = e^{-i\frac{2\pi}{N}}$, so that $W_N^k = e^{-i\frac{2\pi}{N}k}$, which we see in the diagram - this is just the term in front of $O[k]$ from our algebra.

**Summarizing, the DIT FFT algorithm you're implementing is:**
1. Split $x$ into the even and odd indexed components $e$ - this stands for even, not 2.718... - (holding $x[0], x[2], x[4], ...$) and $o$ (holding $x[1], x[3], x[5], ...$), each of length $N / 2$ (we assume $N$ is a power of two).
2. Take the $N/2$ point FFTs (recursively) of these, call them $E$ and $O$.
3. Compute $X[0], ..., X[N/2 - 1]$ via $X[k] = E[k] + e^{-i\frac{2\pi}{N}k} O[k]$. Note that $E[k], O[k]$ are both $N/2$ periodic, so all that actually changes between computing the first and second half of $X[k]$ is the weight applied to $O[k]$.

The DIT FFT algorithm we've presented here is known as the *radix-2* FFT algorithm, because we solve two recursive subproblems of size $N / 2$. In doing this, we assume that our input length is a power of two. This is done so that we can keep dividing our problem size by two until we reach 1, our base case, without worrying about divisibility issues. For example, a length 14 signal would be split into two length 7 signals, which can't be divided in two evenly. 

There are FFT algorithms for other radices, such as 3 (we would assume our input length to be a power of 3, and perform a similar derivation based on that), 5, etc. The state of the art FFT library functions (such as numpy's) use a *mixed radix* FFT, combining several different radices in one, to give an efficient implementation with minimal assumptions about the length. In this lab, we only consider the radix-2 case.

# Your Task: FFT Code

Implement the DIT FFT algorithm in the function `my_fft`. If the length of the input array `x` is not a power of two, just call `dft` on it. You may use the provided `is_power_of_two` function, if you find it helpful, although you certainly are not required to use it or understand how it works.

**Hint 1:** The FFT is a recursive algorithm. Think about what your base case should be. (What is the DFT of a length $N = 1$ sequence?)

**Hint 2:** [Python's slicing features](https://stackoverflow.com/questions/509211/understanding-slice-notation) will be very useful for grabbing the even and odd-indexed components of the signal. 

In [ ]:
def is_power_of_two(n):
    """Return if n is a power of 2, assuming n is a non-negative integer."""
    return n & (n - 1) == 0

In [ ]:
def my_fft(x):
    """
    Decimation in time FFT algorithm.
    """
    N = len(x)
    X = np.zeros(N, dtype=np.complex128)
    
    ## TODO ##

    ## TODO ##
    
    return X

Run the tests to verify correctness. If you implemented `my_fft` correctly, all 5 tests should be passing, and should complete almost instantly. Here, you'll be graded on both speed and correctness - as long as your code completes in under a few seconds, you're fine (the staff solutions takes about 1/4 of a second for all the tests to finish). If both of these aren't true, your implementation isn't correct. Things to check:
- *(Functionality)* Remember, $E[k]$ and $O[k]$ only have $N/2$ elements, so you need to account for the reuse of their terms/wraparound somehow. There are many different approaches.
- *(Functionality)* Just like in `dft`, make sure whatever array you use for the output, $X[k]$, is initialized to be of type `complex128` so you don't discard imaginary parts!
- *(Speed)* Make sure your condition for checking if the length of x is a power of 2 is correct; you may be incorrectly calling `dft` when you don't need to.

In [ ]:
run_fft_tests(my_fft)

# Q3: Performance Profiling

Now that we've got all our functions working, let's actually plot how well they do against each other, as well as numpy's `fft` function. We'll do this by analyzing:
- Our naive DFT and DIT FFT against numpy's FFT, with zero-padding to the next power of 2. 
- Our naive DFT and DIT FFT against numpy's FFT, *without* zero-padding. 
- Numpy's FFT on its own, to get a peek into the state of the art in FFT implementations.

Throughout this process we will stick to a few conventions to be consistent:
1. Average each function's runtime for a signle input size over 20 trials. This is enough so that we reduce any measurement noise from busy background tasks without taking insanely long to run.
2. Only test input sizes from 1 to 512 (with an exception on part b, where we'll do up to $3^7$). We want to crank it up high enough so that DFT is noticeably bad, and that we can tell the difference between our function and numpy's, but not so much that it takes forever to run.

**For this question, the code has been written for you. Your job is to run the cells and interpret the results.**

In [ ]:
INPUT_SIZES = [x for x in range(1, 513)]     # 1, 2, 3, 4, ..., 512
NUM_TRIALS = 20

In [ ]:
def time_execution(f, arg, num_trials):
    """
    Returns the runtime of a single argument function f when called on arg,
    averaged over num_trials trials.
    """
    times = []
    for _ in range(num_trials):
        t0 = time.time()
        f(arg)
        tf = time.time()
        times.append(tf - t0)
    return sum(times) / len(times)

In [ ]:
def next_power_of(k, n):
    """
    Returns the next power of k after and including the number n.
    Not numerically stable.
    
    >>> next_power_of(2, 5)     # next power of 2 after/including 5
    8
    >>> next_power_of(2, 16)    # next power of 2 after/including 16
    16
    >>> next_power_of(3, 81)    # next power of 3 after/including 81
    81
    >>> next_power_of(3, 28)    # next power of 3 after/including 28
    81
    """
    res = np.ceil(log(n, k))
    return k ** res.astype('int')

In [ ]:
def get_avg_runtimes(f, input_sizes, num_trials, zero_pad=False):
    """
    Given an FFT function f and an iterable type (list, range, etc.) of input_sizes,
    returns the average (over num_trials) trials runtime of f for randomly generated 
    data of each size within input_sizes.
    """
    runtimes = []
    for n in input_sizes:
        data = random_complex_array(n)
        if zero_pad:
            L = next_power_of(2, len(data)) - len(data)
            data = np.concatenate((data, np.zeros(L)))
        runtimes.append(time_execution(f, data, num_trials))
    return runtimes

In [ ]:
def plot_runtimes(input_sizes, dft_avg_times, my_avg_times, numpy_avg_times, plot_title, ymax=None):
    plt.figure(figsize=(16, 4))
    plt.title(plot_title)
    plt.plot(input_sizes, dft_avg_times)
    plt.plot(input_sizes, my_avg_times)
    plt.plot(input_sizes, numpy_avg_times)
    plt.ylabel("Average Runtime (seconds)")
    plt.xlabel("Input Size")
    plt.legend(("Naive DFT", "Our FFT", "Numpy's FFT"))
    plt.xlim([1, input_sizes[-1]])
    if ymax:
        plt.ylim([0, ymax])
    plt.show()

In [ ]:
def plot_runtimes_2(input_sizes, avg_times, plot_title, ymax=None):
    plt.figure(figsize=(16, 10))
    plt.title(plot_title)
    plt.plot(input_sizes, avg_times)
    plt.ylabel("Average Runtime (seconds)")
    plt.xlabel("Input Size")
    plt.xlim([1, input_sizes[-1]])
    if ymax:
        plt.ylim([0, ymax])
    plt.show()

Before we get started on the individual analyses, note that by far the most computationally expensive thing we'll have to do is generate the average runtimes for `dft`. Since we only need to do that once, let's do it here, and store it in a variable `dft_runtimes` so that we can reuse it later. 

**The cell will take a long time to run - roughly 20-30 minutes. You can leave this going on in the background of your computer while you do something else, but do NOT shut your computer screen or it may mess with your data taking. If you want, after getting the data once, you can print out the array and save it in the notebook by putting it inside a Python comment with `# my times here`.**

In [ ]:
dft_avg_times = get_avg_runtimes(dft, INPUT_SIZES, NUM_TRIALS)

# Q3a: Performance Without Zero-Padding

Run the cell below to compare the performance of your function against the naive DFT. For now, we won't zero pad.

**The cell will take a long time to run - roughly 20-30 minutes. You can leave this going on in the background of your computer while you do something else, but do NOT shut your computer screen or it may mess with your data taking. If you want, after getting the data once, you can print out the array and save it in the notebook by putting it inside a Python comment with `#`.**

In [ ]:
my_times_3a = get_avg_runtimes(my_fft, INPUT_SIZES, NUM_TRIALS, zero_pad=False)
numpy_times_3a = get_avg_runtimes(np.fft.fft, INPUT_SIZES, NUM_TRIALS, zero_pad=False)

In [ ]:
title = "Runtimes for Non Zero-Padded Inputs"
plot_runtimes(INPUT_SIZES, dft_avg_times, my_times_3a, numpy_times_3a, title)

**Q: You should see the "Naive DFT" time and "Our FFT" times more or less overlapping, with a few distinct spikes where "Our FFT" does really well. Explain why the graph looks like this.**

<span style="color:blue">A: (TODO)</span>

# Q3b: Performance With Zero-Padding

Run the cells below to profile your FFT function; this time we'll pad everything to the next power of 2. This will not take as long to run - it should only be a few minutes.

In [ ]:
my_times_3b = get_avg_runtimes(my_fft, INPUT_SIZES, NUM_TRIALS, zero_pad=True)
numpy_times_3b = get_avg_runtimes(np.fft.fft, INPUT_SIZES, NUM_TRIALS, zero_pad=True)

In [ ]:
title = "Runtimes for Zero-Padded Inputs"
plot_runtimes(INPUT_SIZES, dft_avg_times, my_times_3b, numpy_times_3b, title)

**Q: Now, we seem to be doing a lot better. Why does zero-padding help?**

<span style="color:blue">A: (TODO)</span>

**Q: The "Our FFT" plot should look piecewise constant. It's still asymptotically upper bounded by $N \log N$, but why is it piecewise constant rather than some scaled version of the function $N \log N$?** (Hint: Are the runtimes for inputs of size 255 and 256 the same? Why or why not? What about all inputs in the range 129, 130, 131, ..., 256?)

<span style="color:blue">A: (TODO)</span>

# Q3c: Profiling `numpy.fft.fft`

Now that we've played around with our own function, let's do numpy's!

Since numpy's is much faster than our other functions, and it's hard to see the relevant features for small inputs, we'll crank up both the number of trials AND the range of input sizes here. We'll also use a taller plot.

For Q3c, you just have to run the cells; there's no code to write or conceptual questions to answer.

In [ ]:
numpy_inputs = [x for x in range(1, 1025)]
numpy_times = get_avg_runtimes(np.fft.fft, numpy_inputs, num_trials=100, zero_pad=False)

In [ ]:
title = "Average Runtimes For numpy's FFT Implementation"
plot_runtimes_2(numpy_inputs, numpy_times, title)

### What's up with the graph above?
- As discussed above, `numpy.fft.fft` uses a *mixed-radix* approach, combining FFT implementations specialized for powers of different integers so that it can efficiently compute the DFT of inputs of various lengths, not just powers of 2.
- This mixed radix approach is a large reason for these spikes:
    - The lowest points (i.e. fastest running input sizes) correspond to powers of 2 or 3, as these are easily handled by radix-2 and radix-3 algorithms. 
    - The maxima correspond to prime numbers - for example, 599, 601, and 607 are all prime numbers; it's hard to make out the exact x-axis values, but we can certainly see some peaks around 600. There's no way to decompose these into smaller primes that we have fixed-radix (e.g. radix-2, radix-3) algorithms to help with.
- You'll also see that the actual time in seconds this function takes is very low compared to some of the numbers we were looking at earlier - in addition to the mixed-radix approach, `numpy.fft.fft`'s parent library is FFTW, Fastest Fourier Transform in the West, a FORTRAN and C based numerical library that's been around since the 1970s, with every performance optimization trick you can imagine baked in.

# Q4: Building a Heart Rate Detector with a Cell Phone Camera

One of the most important vital signs for human beings is their heart rate — the number of times their heart beats in a minute. For a human adult, a typical resting heart rate is in the range of 60-100 beats per minute. Typically, more athletic people have higher average resting heart rates. For instance, very athletic people may have a heart rate closer to 40 or 50 beats per minute.

Generally, heart rate monitoring is done through high cost devices with a system of sensors designed specifically for that purpose. In this question, we'll make a heart rate detector with just Python and a cell phone camera, courtesy of the Fourier Transform.

Our detector will be based on the fact that your heart rate drives the flow of blood through your body. If we take a close-up video recording of a person's thumb — other body parts will work too, but it's easiest to just hold your thumb over your cell phone camera — and only consider the red channel (each image or "frame" of a video encodes Red, Green, and Blue (RGB) value) of the recording, it will give us information about the blood flow through that region of their body during the recording. However, this data is a video, which is three-dimensional (one for time to select which frame/image you want, one to select the image's x-axis pixel, one to the select the image's y-axis pixel; the signal value is the pixel intensity), and we only know how to work with 1D signals in EE 120. 

## Converting a Video to a 1D Signal

To solve this problem, we *spatially average* the video: at each point in time, we take the image and use the average intensity of the red (R) channel across all pixels as our signal value. The resultant signal is 1D, giving the average intensity of the red versus time.

**The data is provided for you for this question.** It's a 30 second video of a thumb held up close to an iPhone camera.

Run the cell below to load the video and plot one frame of it (an image). You don't have to understand any of this code - it's just handling the video processing. 

It's worth watching the video (just open it from your directory) on your own to see the thumb pulsing - you can actually see the heartbeats in the original video!

In [ ]:
videodata = skvideo.io.vread("finger.mov")  
plt.imshow(videodata[238, :, :, :]) # plot one frame

If the video loaded correctly, you should see a picture that looks similar to a color gradient from orange to black as you look left to right. If you're unsure, you can ask in office hours or on Piazza. This isn't meant to be any sort of trick question.

Once the video finishes loading, run the next cell to extract the red component, spatially average it, and plot it.

In [ ]:
# Extract red component
red_vals = videodata[:, :, :, 0]

# Spatially average
avg_red = red_vals.mean(axis=(1, 2)) # average along both axes
plt.figure(figsize=(10, 4))
plt.plot(avg_red, 'r')
plt.ylabel("Average Red Intensity")
plt.xlabel("Time")
plt.show()

There's two important features of this plot that are worth explaining:
1. There signal mostly jumps up and down - this corresponds to the heart actually beating! Note that this is (more or less) periodic, as your resting heart rate should be fairly consistent, ignoring external stimuli.
2. There's a huge jump in the middle. This is because the person taking the video moved their thumb during the recording. Fortunately, it won't affect our analysis. Ideally, we'd retake the data, but it's good to be aware of these kinds of issues, as they often arise when correcting real world data.

## Detrending

There's one more thing we need to do before trying to determine the person's heart rate from this spatially averaged recording: *detrend* the signal. In many contexts, signals have *trends*, or general tendencies in the directions they move. For example, a stock that is doing well may have an upward trend. Many trends are linear, or well-approximated as linear over a sufficiently short interval. 

Here's a visual example of linear detrending of a stock, taken from Reference 6:

![](detrend.png)

If you're familiar with "demeaning" to set a signal's average value to zero by subtracting its mean from it (perhaps from a probability course), this is similar to that, except that detrending subtracts an entire "mean line", rather than just one scalar value as is done when you de-mean a signal (or random variable).

Looking at the plot of our red signal, we can see a linear trend of decay, likely due to the person not keeping their thumb at the same distance from the camera during the entire recording. Much like the jump in the middle of the signal, this is an artifact of our data, rather than a feature we want to analyze.

Fortunately, scipy has a nice function for removing linear trends from signals: `scipy.signal.detrend`. It works by calculating the signal's linear trend, defined as the best-fit line through the signal (this is done using the least-squares fit you learned about in EE 16A), and subtracting it from the signal.


Run the cell below to detrend the signal and plot the result along with the original.

In [ ]:
avg_red_detrend = signal.detrend(avg_red)

plt.figure(figsize=(12, 10))
plt.plot(avg_red, 'r')
plt.plot(avg_red_detrend, 'b')
plt.ylabel("Average Red Channel Intensity")
plt.xlabel("Sample Index")
plt.xlim((0, len(avg_red)))
plt.legend(("Average Red Value", "Average Red Value (Detrended)"))
plt.show()

Now we're all ready to measure heart rate!

## Frequency Domain Analysis to Find the Heart Rate

In EE 120, we mainly discuss the Fourier Transform as a tool for analyzing LTI Systems. However, it's even more widely used in practice for analyzing the frequency contents of signals. 

As we saw in the earlier parts of the lab, we can use the FFT to efficiently compute the DFT, which gives us samples of a signal's DTFT. This allows us to understand the strengths of the oscillations that make up the signal, so to speak. In the case of our data, this interpretation is very natural - our signal has fluctuations up and down corresponding to the person's heart beating, so the signal's *dominant frequency* (the frequency at which the magnitude of the signal's DTFT is maximized) is precisely the frequency at which their heart is beating! Recall that the DTFT is complex-valued, meaning it contains information about both magnitude and phase. Intuitively, magnitude corresponds to the "strength" of the frequency whereas phase enocdes information about where the frequency is present in the signal. Since we assume an approximately constant heart rate throughout the recording, we only care about the strengths of the frequencies, not where they show up in the signal.

We'll use this idea of finding the dominant frequency to compute the person's estimated heart rate. 

### A Sample of Sampling Theory

The data we have here (the video) was obtained by a cell phone camera taking pictures at 30 frames per second. This means we get one new sample of data (i.e., an image of the thumb) 30 times each second. As a result, we say that our *sampling frequency* is 30 Hz for this data. Alternatively, the data was *sampled at 30 Hz*.

We haven't covered sampling in detail yet in EE 120. Fortunately, there's only one result relevant to our analysis here: the Shannon-Nyquist Sampling Theorem. Briefly speaking, it says that if you take uniformly spaced (i.e. the time that elapses between each sample you take is the same - we do not discuss nonuniform sampling in EE 120) samples of a signal at $f$ Hz, the highest frequency you will observe corresponds to $f / 2$ Hz. The implication of this when taking the FFT of our data (again, this is just a sampling of the DTFT) is that the x-axis, which implicitly goes from $-\pi$ to $\pi$ (any interval of length $2\pi$ will do since the DTFT is $2\pi$-periodic, but this is the most popular choice), phyiscally corresponds to going from -15 Hz to 15 Hz (since the highest frequency is $\pi$ in discrete-time, and the highest frequency we can make out is 15 Hz). As a result, when plotting the FFT, it's useful to use x-axis labels ranging from -15 to 15.

Run the cell below to compute and plot the magnitude detrended signal's spectrum, using the FFT.

In [ ]:
reds_windowed = np.hamming(len(avg_red_detrend)) * avg_red_detrend
reds_FFT = np.fft.fftshift(np.fft.fft(reds_windowed, 1024))
w = np.linspace(-15, 15, len(reds_FFT))
plt.figure(figsize=(16, 4))
plt.plot(w, abs(reds_FFT), 'r')
plt.ylabel("Magnitude")
plt.xlabel("Frequency (Hz)")
plt.show()

This plot shows the entire spectrum, -15 Hz to 15 Hz. We only care about frequencies that make sense for what the human heart can produce. Since Hz is a measure of how many times something happens in a second, we have to multiply by 60 to obtain the number of occurrences per minute (i.e. heart rate). Let's ignore negative frequencies for now, and turn our attention to the 0 to 15 Hz range. 

Multiplying by 60, we see that 15 Hz would correspond to 900 beats per minute, which is out of the question. We should only be concerned with finding the dominant frequency over a realistic range. In beats per minute, anything between 40 (you're a competitive athlete and currently on the couch watching TV) and 200 (you're on an intense run) is reasonable. Dividing by 60 to convert to Hz, this gives us the range of .67 Hz to 3.4 Hz.

Run the cell below to crop the frequency indices and the FFT, and plot the spectrum over this range.

In [ ]:
# We did the math for you - we want the indices 495 to 568
w_cropped = w[535:628]
reds_FFT_cropped = reds_FFT[535:628]

# Plot
plt.figure(figsize=(16, 4))
plt.plot(w_cropped, abs(reds_FFT_cropped), 'r')
plt.ylabel("Magnitude")
plt.xlabel("Frequency (Hz)")
plt.show()

Now, the final step: determining the heart rate. This time, it's your turn. Using the method we discussed above of picking the dominant frequency, find the person's heart rate. Store your result in the variable `bpm`.

**Hint 1:** The result you want to get is a frequency, i.e. an element in `w_cropped`, rather than some value in the FFT. After finding it, multiply by 60, since `w_cropped` contains values in Hz.  
**Hint 2:** The function `np.argmax` will return the index of the maximum value of an array. For example, `np.argmax(np.array([4, -5, 0, 17, 2]))` would return 3, as the largest element is the fourth one (which has index 3 in zero-indexing).  
**Hint 3**: Make sure you use the absolute value of `reds_FFT_cropped`, not just `reds_FFT_cropped`, when finding the maximum.

In [ ]:
# TODO your heart rate code


In [ ]:
# TODO run me after filling in above cell
print("Heart Rate: {0} beats per minute".format(round(bpm, 1)))

While taking data, we independently measured the person's heart rate to be around 74-77 beats per minute. How do your results compare? (You don't have to write out an answer here; just use this as a sanity check on your result)

### Final Words
- The post-processing methods we used here are not very robust, but work for illustrating the general principle of heart rate monitoring via the FFT. For example, using a single point maximum as a criterion is prone to issues with outliers. If you think it's fun to consider ways to improve making this procedure more robust (feel free to experiment in a separate notebook with your own data and try it out!), consider taking EE 123, Digital Signal Processing.
- With a bit of work, you could turn this question into a real-time heart rate detector.  With the appropriate hardware configurations and Python libraries, you could obtain real-time video of someone's thumb, arm, etc., and run this algorithm in a loop to constantly output the heart rate it detects after averaging the data it collects every few seconds.
- We used 30 seconds of video data rather than a shorter 5-10 second clip - this results in a larger file size, but it's necessary to help combat the noise - for statistical reasons, given certain assumptions about your noise, as you average a noisy signal over more and more samples, which is essentially what we do by considering the FFT of the whole R-component averaged video, your noise cancels itself out more and more. 
    - This is exactly why if you go to the RSF, for example, and try to measure your heart rate on a treadmill there, you'll have to wait several seconds before it gives you a reading - it's not using a video recorder to determine your heart rate, but the same exact principle of it needing enough samples to average out the noise applies!

# References

[1] Video lecture on the FFT. [Link](https://www.youtube.com/watch?v=1mVbZLHLaf0)   
[2] Wikipedia page on FFT. Has a great derivation of the algorithm. [Link](https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm)  
[3] Notes from MIT's 6.341 (Discrete-Time Signal Processing) on the FFT. [Link](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-341-discrete-time-signal-processing-fall-2005/lecture-notes/lec19.pdf)  
[4] Lecture slides from EE 123 Spring 2018 on FFT. [Link 1](https://inst.eecs.berkeley.edu/~ee123/sp18/Notes/Lecture3C.pdf), [Link 2](https://inst.eecs.berkeley.edu/~ee123/sp18/Notes/Lecture4A.pdf)  
[5] Modern uses of DFT. [Link](http://news.mit.edu/2009/explained-fourier)  
[6] Visual example of detrending. [Link](https://www.mathworks.com/help/matlab/data_analysis/detrending-data.html)  
[7] Finding heart rate via the FFT. [Link](http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/6796/pdf/imm6796.pdf)